In [ ]:
import optuna
import numpy as np

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01, epochs=500):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        self.epochs = epochs
        
        self.weights_input_hidden = np.random.randn(self.input_size, self.hidden_size) * 0.1
        self.bias_hidden = np.zeros((1, self.hidden_size))
        self.weights_hidden_output = np.random.randn(self.hidden_size, self.output_size) * 0.1
        self.bias_output = np.zeros((1, self.output_size))
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def sigmoid_derivative(self, z):
        return z * (1 - z)
    
    def fit(self, X, y):
        for _ in range(self.epochs):
            hidden_layer_activation = np.dot(X, self.weights_input_hidden) + self.bias_hidden
            hidden_layer_output = self.sigmoid(hidden_layer_activation)
            final_layer_activation = np.dot(hidden_layer_output, self.weights_hidden_output) + self.bias_output
            output = self.sigmoid(final_layer_activation)
            
            error = y - output
            
            d_output = error * self.sigmoid_derivative(output)
            d_hidden_layer = np.dot(d_output, self.weights_hidden_output.T) * self.sigmoid_derivative(hidden_layer_output)
            
            self.weights_hidden_output += np.dot(hidden_layer_output.T, d_output) * self.learning_rate
            self.bias_output += np.sum(d_output, axis=0, keepdims=True) * self.learning_rate
            self.weights_input_hidden += np.dot(X.T, d_hidden_layer) * self.learning_rate
            self.bias_hidden += np.sum(d_hidden_layer, axis=0, keepdims=True) * self.learning_rate
    
    def predict(self, X):
        hidden_layer_activation = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        hidden_layer_output = self.sigmoid(hidden_layer_activation)
        final_layer_activation = np.dot(hidden_layer_output, self.weights_hidden_output) + self.bias_output
        output = self.sigmoid(final_layer_activation)
        return np.round(output)

class LogisticRegression:
    def __init__(self, learning_rate=0.01, epochs=500):
        self.learning_rate = learning_rate
        self.epochs = epochs
    
    def fit(self, X, y):
        self.weights = np.zeros(X.shape[1])
        self.bias = 0
        m = len(y)
        
        for _ in range(self.epochs):
            linear_model = np.dot(X, self.weights) + self.bias
            predictions = 1 / (1 + np.exp(-linear_model))
            
            dw = (1/m) * np.dot(X.T, (predictions - y))
            db = (1/m) * np.sum(predictions - y)
            
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db
    
    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        predictions = 1 / (1 + np.exp(-linear_model))
        return np.round(predictions)


def train_test_split(X, y, test_size=0.2, random_state=None):
    if random_state:
        np.random.seed(random_state)
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    
    split_idx = int(X.shape[0] * (1 - test_size))
    train_indices = indices[:split_idx]
    test_indices = indices[split_idx:]
    
    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]

def accuracy_score(y_true, y_pred):
    return np.mean(y_true == y_pred)

def optimize_model(trial, model_name, X_train, y_train, X_val, y_val):
    if model_name == "NeuralNetwork":
        learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-1)
        hidden_size = trial.suggest_int("hidden_size", 5, 50)
        epochs = trial.suggest_int("epochs", 100, 1000)
        
        model = NeuralNetwork(input_size=X_train.shape[1], hidden_size=hidden_size, output_size=1, learning_rate=learning_rate, epochs=epochs)
    elif model_name == "LogisticRegression":
        learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-1)
        epochs = trial.suggest_int("epochs", 100, 1000)
        
        model = LogisticRegression(learning_rate=learning_rate, epochs=epochs)
    
    model.fit(X_train, y_train)
    predictions = model.predict(X_val)
    return accuracy_score(y_val, predictions)

def auto_ml(X, y, models):
    X, y = preprocess_data(X, y)
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    best_model_name = None
    best_params = None
    best_score = 0
    final_model = None
    
    for model_name in models:
        def objective(trial):
            return optimize_model(trial, model_name, X_train, y_train, X_val, y_val)
        
        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=30)  # 30 essais par modèle
        
        if study.best_value > best_score:
            best_model_name = model_name
            best_params = study.best_params
            best_score = study.best_value
            
            final_model = optimize_model(trial, model_name, X_train, y_train, X_val, y_val)
    
    return final_model, best_model_name, best_params, best_score

if __name__ == "__main__":
    X = np.random.rand(1000, 2)
    y = (X[:, 0] + X[:, 1] > 1).astype(int).reshape(-1, 1)
    
    models = ["NeuralNetwork", "LogisticRegression"]
    
    final_model, best_model_name, best_params, best_score = auto_ml(X, y, models)
    
    print("Meilleur modèle :", best_model_name)
    print("Meilleurs hyperparamètres :", best_params)
    print("Meilleure précision :", best_score)
